# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("Output/weather_data.csv")
cities_df.head(10)


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,0,punta arenas,-53.1500,-70.9167,59.00,55,0,18.41,1613048757
1,1,carnarvon,-24.8667,113.6333,75.20,83,0,24.16,1613048757
2,2,dikson,73.5069,80.5464,-21.24,88,99,4.36,1613048757
3,3,georgetown,5.4112,100.3354,86.00,74,20,6.91,1613048500
4,4,hobart,-42.8794,147.3294,66.00,77,90,9.22,1613048758
5,5,albany,42.6001,-73.9662,21.99,60,100,3.04,1613048758
6,6,simao,22.7886,100.9748,53.67,84,0,3.13,1613048758
7,7,victoria,22.2855,114.1577,64.99,73,41,10.38,1613048758
8,8,souillac,-20.5167,57.5167,84.20,61,40,11.50,1613048759
9,9,saint-philippe,-21.3585,55.7679,87.01,62,0,32.21,1613048759


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_df[['Lat', 'Lng']]
humidity = cities_df['Humidity']
fig = gmaps.figure()
humidity_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 75, point_radius = 1)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Conditions: Avg temperature between 70 and 80 degrees, humidity between 30 and 50%, zero cloudiness
temp_sort1 = cities_df[cities_df['Max Temp'] >=70]
temp_sort2 = cities_df[cities_df['Max Temp'] <= 80]
combined_temp = pd.concat([temp_sort1, temp_sort2], ignore_index = True)
humidity_sort1 = combined_temp[combined_temp['Humidity'] >= 30]
humidity_sort2 = combined_temp[combined_temp['Humidity'] <= 50]
combined_humidity = pd.concat([humidity_sort1, humidity_sort2], ignore_index = True)
ideal_df = combined_humidity[combined_humidity['Cloudiness'] == 0]

ideal_df.dropna()                           

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,1,carnarvon,-24.8667,113.6333,75.20,83,0,24.16,1613048757
3,9,saint-philippe,-21.3585,55.7679,87.01,62,0,32.21,1613048759
7,27,hermanus,-34.4187,19.2345,84.99,59,0,8.01,1613048763
11,34,arraial do cabo,-22.9661,-42.0278,84.20,65,0,14.97,1613048765
19,51,poum,-20.2333,164.0167,78.87,74,0,13.65,1613048768
...,...,...,...,...,...,...,...,...,...
737,444,tura,25.5198,90.2201,64.18,47,0,3.47,1613048845
739,475,zomin,39.9606,68.3958,55.02,46,0,3.06,1613048851
740,512,jalu,29.0331,21.5482,79.45,15,0,7.67,1613048857
741,514,praia,14.9215,-23.5087,78.80,47,0,20.71,1613048858


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
params = {}
params['key'] = g_key
params['rankby'] = 'distance'
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Lat'] = ""
hotel_df['Hotel Lng'] = ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

counter = 0
for index, row in hotel_df.iterrows():
    city_lat = row['Lat']
    city_lng = row['Lng']
    params['location'] = f"{city_lat}, {city_lng}"
    params['type'] = 'hotel'
    try:
        google_response = requests.get(base_url, params = params)
        google_json = google_response.json()
        hotel_df.loc[index, 'Hotel Name'] = google_json['results'][0]['name']
        hotel_df.loc[index, 'Hotel Lat'] = google_json['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Hotel Lng'] = google_json['results'][0]['geometry']['location']['lng']
       
        counter += 1
    except:
        
        print(f"Error on index {counter}")
        
    
    
    




<ipython-input-5-3ad5ec669042>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""
<ipython-input-5-3ad5ec669042>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Lat'] = ""
<ipython-input-5-3ad5ec669042>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

Error on index 39
Error on index 128


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Lat', 'Lng']]

In [7]:
# Add marker layer ontop of heat map
#marker_lat = []
#for index, row in hotel_df.iterrows():
 #   marker_lat.append(hotel_df.loc[index, 'Hotel Lat'])
  #  marker_lng.append(hotel_df.loc[index, 'Hotel Lng'])
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

    

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))